<a href="https://colab.research.google.com/github/Marcosrbsantos/agente-posts-ai/blob/main/notebooks/agente_design_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# 1. Instala Git e clona o repositório
!apt-get update -qq && apt-get install -qq git
!git clone https://github.com/Marcosrbsantos/agente-posts-ai.git
%cd agente-posts-ai

# 2. Instala bibliotecas gratuitas
!pip install --upgrade pip
!pip install diffusers transformers accelerate faiss-cpu sentence-transformers langchain bitsandbytes gradio
!pip install -U langchain-community
!pip install pypdf
!pip install --upgrade pip
!pip install diffusers transformers accelerate faiss-cpu sentence-transformers langchain bitsandbytes gradio langchain-community pypdf





W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Cloning into 'agente-posts-ai'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 27 (delta 9), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 158.33 KiB | 4.80 MiB/s, done.
Resolving deltas: 100% (9/9), done.
/content/agente-posts-ai/agente-posts-ai/agente-posts-ai/agente-posts-ai/agente-posts-ai


In [13]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("Guia Técnico de Design Gráfico para Posts em Mídias Sociais.pdf")
docs   = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
chunks = splitter.split_documents(docs)
print("Chunks criados:", len(chunks))



Chunks criados: 64


In [16]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS

# Usando o wrapper do LangChain para Sentence-Transformers
embed_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embed_model)


<ipython-input-16-34be51aacbf8>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

# Carrega Falcon-7B-Instruct em 8-bit
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct", trust_remote_code=True)
model     = AutoModelForCausalLM.from_pretrained(
    "tiiuae/falcon-7b-instruct",
    device_map="auto",
    load_in_8bit=True
)
hf_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)
llm     = HuggingFacePipeline(pipeline=hf_pipe)

# Monta o chain RAG
retriever = vectorstore.as_retriever(search_kwargs={"k":3})
qa        = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [ ]:
pergunta = "Como organizar camadas para um post de Instagram?"
resposta = qa.run(pergunta)
print("Pergunta:", pergunta)
print("Resposta:", resposta)


In [18]:
import gradio as gr

def gerar_design(briefing: str):
    # 1) extrai spec via RAG
    spec = {
        "layers": qa.run("Explique a ordem de camadas para um post."),
        "colors": qa.run("Quais esquemas de cores complementares?"),
        "typography": qa.run("Quais regras tipográficas usar?")
    }
    # 2) monta prompt de difusão (exemplo simples)
    prompt = f"{spec['layers']} {spec['colors']} {spec['typography']} {briefing}"
    # 3) gera imagem
    from diffusers import StableDiffusionPipeline
    pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype="auto"
    ).to("cuda")
    img = pipe(prompt, num_inference_steps=25, guidance_scale=7.5).images[0]
    return img

gr.Interface(
    fn=gerar_design,
    inputs="text",
    outputs="image",
    title="Agente de Design"
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1cd97a2ac4899f2f5f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
